In [1]:
import pandas as pd 
from pathlib import Path
import os
import numpy as np

In [2]:
cd ..

c:\Gan\GAN-market-prediction


In [3]:
stock_path = Path(os.path.abspath("")) / "data" / "stock_data"
save_path = Path(os.path.abspath("")) / "paper"
tickers = ["EA", "UBSFY", "TTWO", "ATVI"]

In [4]:
df = pd.DataFrame()
for ticker in tickers:
    tmp = pd.read_csv(stock_path / f"{ticker}.csv").tail(178)
    tmp["Company"] = ticker
    df = df.append(tmp)

C:\Users\kuba1\AppData\Local\Temp\ipykernel_20580\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_20580\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_20580\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_20580\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


In [52]:
df.groupby(by="Company")

Company
ATVI      83.648342
EA       139.189536
TTWO     170.877192
UBSFY     12.836576
Name: Close, dtype: float64

In [12]:
pd.options.display.float_format = '{:.5f}'.format

In [11]:
(
    df.loc[:, [col for col in df.columns if col not in ["Dividends", "Stock Splits"]]]
    .groupby(by="Company")
    .describe()
    .T
)


TypeError: cannot convert the series to <class 'float'>

In [44]:
with open(save_path / "desc_stock.tex", "w") as f:
    f.write(
        df.loc[
            :, [col for col in df.columns if col not in ["Dividends", "Stock Splits"]]
        ]
        .groupby(by="Company")
        .describe()
        .T.to_latex(caption="Descriptive statistics for chosen companies")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\838483005.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.loc[


In [50]:
results_dict = {
    "ATVI": [1.3585, 1.8867],
    "EA": [1.3427, 1.8956], 
    "TTWO": [3.5405, 4.5069],
    "UBSFY": [0.2578, 0.3376]
}

In [53]:
with open(save_path / "metrics_err.tex", "w") as f:
    f.write(
        pd.DataFrame(results_dict)
        .append(df.groupby(by="Company")["Close"].mean())
        .rename(index={0: "MAE", 1: "RMSE", "Close": "Close Price Mean"})
        .to_latex(caption="Error metrics for chosen companies")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\1653263745.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd.DataFrame(results_dict)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\1653263745.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pd.DataFrame(results_dict)


In [47]:
errs = pd.DataFrame(results_dict).append(df.groupby(by="Company")['Close'].mean()).rename(index={0: "MAE", 1:"RMSE", "Close": "Close Price Mean"})

C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\3186659599.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errs = pd.DataFrame(results_dict).append(df.groupby(by="Company")['Close'].mean()).rename(index={0: "MAE", 1:"RMSE", "Close": "Close Price Mean"})


In [54]:
errs.T

,MAE,RMSE,Close Price Mean
ATVI,1.3585,1.8867,83.648342
EA,1.3427,1.8956,139.189536
TTWO,3.5405,4.5069,170.877192
UBSFY,0.2578,0.3376,12.836576


In [49]:
(errs.T['MAE'] / errs.T['Close Price Mean']) * 100

ATVI     1.624061
EA       0.964656
TTWO     2.071956
UBSFY    2.008324
dtype: float64

In [21]:
hparams = {
    "top_cut_off": np.arange(0, 4, 0.2),
    "down_cut_off": np.arange(0, 4, 0.2),
    "if_short": [True, False] + 18 * [None],
}

In [22]:
hparams_df = pd.DataFrame(hparams)

In [24]:
hparams_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
top_cut_off,0.0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2,3.4,3.6,3.8
down_cut_off,0.0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2,3.4,3.6,3.8
if_short,True,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [28]:
with open(save_path / "signals_hpars.tex", "w") as f:
    f.write(
        hparams_df
        .to_latex(caption="Signal hyper parameters")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\2764230600.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  hparams_df
